I've been given a new task to study how scale dependent bias varias as a function of HOD params and 

In [1]:
from pearce.mocks.kittens import cat_dict
import numpy as np
from scipy.stats import binned_statistic, linregress

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [3]:
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [4]:
cosmo_params = {'simname':'chinchilla', 'Lbox':400.0, 'scale_factors':[0.81120]}

In [5]:
cat = cat_dict[cosmo_params['simname']](**cosmo_params)#construct the specified catalog!

In [6]:
cat.load(0.81120, tol = 0.01, HOD='redMagic', particles = True)#, hod_kwargs = {'sec_haloprop_key':'halo_log_nfw_conc'})#, hod_kwargs={'split': 0.5})

In [7]:
chain_fname = '/u/ki/swmclau2/des/PearceMCMC/200_walkers_50000_steps_chain_redmagic_bias_z0.23.npy'
chain = np.genfromtxt(chain_fname)

In [8]:
n_walkers = 200
n_params = chain.shape[1]
n_burn = 30000
chain = chain[n_walkers*n_burn:, :]
print chain.shape

(4000000, 6)


In [9]:
ordered_param_names = param_names = ['logMmin', 'f_c', 'logM0', 'sigma_logM',  'logM1',   'alpha']

In [10]:
rbins = np.logspace(-1,1.5,15)
rbc = (rbins[1:]+rbins[:-1])/2

In [11]:
xis = []
indicies = np.random.choice(chain.shape[0], size = 1000, replace = False)
for i, row in enumerate(chain[indicies]):
    print i
    try:
        hod_params = dict(zip(ordered_param_names, row))
        cat.populate(hod_params)
        xi = cat.calc_xi(rbins, use_corrfunc = False, do_jackknife=False)
        xis.append(xi)
    except MemoryError:
        continue
    #plt.plot(rbc, bias, alpha = 0.1, color = 'b')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [ ]:
for xi in xis:
        plt.plot(rbc, xi, alpha = 0.01, color = 'g')

plt.loglog()
plt.xlabel(r'$r$ [Mpc]')
plt.ylabel(r'$\xi_{gg}(r)$')
#plt.ylim([-5, 20])
plt.xlim([1e0, 40])
plt.title(r"Allowed biases from Buzzard Y3 $w(\theta)$")
plt.show()

In [12]:
xi_mm = cat.calc_xi_mm(rbins)

/u/ki/swmclau2/.conda/envs/hodemulator/lib/python2.7/site-packages/halotools-0.6.dev4681-py2.7-linux-x86_64.egg/halotools/mock_observables/two_point_clustering/clustering_helpers.py:134: UserWarning: 
 `sample1` exceeds `max_sample_size` 
downsampling `sample1`...
  warn(msg)


OSError: [Errno 12] Cannot allocate memory

In [ ]:
plt.plot(rbc, xi_mm)
plt.plot(rbc, xi)
plt.loglog();

In [ ]:
for xi in xis:
        plt.plot(rbc, xi/xi_mm, alpha = 0.01, color = 'b')

plt.xscale('log')
plt.xlabel(r'$r$ [Mpc]')
plt.ylabel(r'$\xi_{gg}/\xi_{mm}(r)$')
#plt.ylim([-5, 20])
plt.xlim([1e0, 40])
plt.title(r"Allowed biases from AB chains")
plt.show()

In [ ]:
xis = np.array(xis)
xis.shape

In [ ]:
plt.hist(np.sqrt(xis[:,-1]/xi_mm[-1]), color = 'g', bins =10, normed = True);
plt.xlabel(r"$b$")
plt.ylabel(r"P($b$)")
plt.title("Posterior histrogram of linear biases AB chain")

colors = sns.diverging_palette(80, 190,l= 80, n=N)
sns.palplot(colors)